This notebook requires **cpnest**; see https://github.com/johnveitch/cpnest. 

Can be installed via `pip install cpnest`.

In [ ]:
import os

import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

from corner import corner

import sys
sys.path.append(os.path.abspath('code'))

import cpnest
import cpnest.model

# Introduction to Bayesian Analysis

### Christopher J. Moore 

In this session I will give an introduction to some Bayesian methods.
The focus will be on the statistical ideas, not the physics. Therefore we will work with a highly simplified toy problem and idealised mock data.

**Contents:**
 - **1. A New Class of Transient**: This section describes the toy problem, the mock data and the models we will be using.
 - **2. Analysing Individual Events**: This section applyies Bayesian analysis to the analysis of an individual transient light curve: detection, parameter estimation, and model selection are all discussed. 
 - **3. Hierarchical Bayesian Analysis**: Given several light curves we can start asking questions about the underlying population. This section analyses the population of several events in a mock catalog
 - **4. Conclusions**
 

# Section 1: A New Class of Transient

Supppose that we observe some new type of transient event. We want to analyse a small catalog containing the light curves from the first such events.

## Section 1.1: Data

The light curve data for the small catalog of transietns is stored in *data/catalog.dat*.

In [ ]:
with open("data/catalog.dat") as f:
    for i in range(7): # print first few lines
        line = f.readline()[0:-1]
        
        # Read the error from the header 
        if 'sigma' in line: sigma = float(line.split()[-1])
        
        print(line)

**Column 0:** time stamps. (For simplicity, times are shifted such that peak occurs at $t=0$ and resampled at 1Hz.)

**Columns 1 to Nevents:** light curves for first few events. (Each flux measurement is assumed to have an identical, independent Gaussian error which is given in the file.)


In [ ]:
catalog_data = np.loadtxt("data/catalog.dat")

times = catalog_data[:,0]
Nevents = catalog_data.shape[1] - 1

for N in range(Nevents):
    plt.plot(times, catalog_data[:,N+1], label='event '+str(N))  
    
plt.xlabel("Time [s]")
plt.ylabel("Flux [arb units]")

plt.legend(loc='upper right')
plt.show()

### 1.1: Model(s)

The simplest possible situation is that there is no signal present in the data (null hypothesis).

**Model 0** is the null hypothesis,
$$ F_0(t) = 0 $$

Now suppose that there are two competing models for the light curve signals. 

**Model A** is a Gaussian, 
$$ F_A(t) = A\exp\left( \frac{-t^2}{\tau^2} \right) \,.$$

**Model B** has a similar shape but with broader tails,
$$ F_B(t) = \frac{A \tau^2}{\tau^2+t^2 } \,. $$

Both signal hypotheses have two parameters: an amplitude $A$, and a duration $\tau$; we collectively denote the model parameters
$\theta^{\mu} = (A, \tau)$.

These models are coded up in *Models.py*; let's import and inspect these functions.

In [ ]:
from Models import ModelA_lightcurve as ModelA
from Models import ModelB_lightcurve as ModelB

print('def ModelA(times, params):\n    """', ModelA.__doc__[:-1], '"""')

# Section 2.0: Analysing Individual Events 

## Section 2.1: Bayes' Theorem

Bayes' theorem is the basis for everything we do in the this notebook;
$$ P(\theta^{\mu} | \mathrm{data}) = P(\mathrm{data} | \theta^{\mu}) \frac{P(\theta^{\mu}_{i})}{P(\mathrm{data})} \,. $$
This equation is to be understood as appling to a particular event; i.e. $\theta=\theta_i$ for $i\in\{1,\ldots, N_\rm{events}\}$ and $\mathrm{data}=\mathrm{data}_i$. 

Each term has a special name:

#### The Posterior: $P(\theta| \rm{data} )$

This is what we want to find (sometimes known as the target distribution). It is the probability of the model parameters given the data.

#### The Likelihood: $P(\rm{data} | \theta) \equiv \mathcal{L}(\theta)$

This is a input to the Bayesian analysis.

The probability of the data given a particular set of model parameters.

For our toy problem, because we have assumed simple noise (i.e. identical, independent Gaussian etc) the likelihood takes a simple form;
$$ \log\mathcal{L}(\theta) = \frac{-1}{2}\sum_{m=1}^{N_\mathrm{times}}\frac{\big[\mathrm{data}_m-\mathrm{model}(\theta)_m)\big]^2}{\sigma^{2}} -\frac{N_\mathrm{times}}{2}\log(2\pi\sigma^2) \,. $$

#### The Prior: $P(\theta)\equiv \Pi(\theta)$

This is a input to the Bayesian analysis.

The prior probability on the source parameters. There are no concrete rules for how to choose priors. 
However, a reasonable choice here would be a seperable prior on the two parameters; i.e. 
$$\Pi(\theta^{\mu})=\Pi(A)\Pi(\tau)\,.$$
For the amplitude the prior is uniform in log (Jeffreys 
prior) between lower/upper cutoffs $A_\mathrm{lower}=0.03$ and $A_\mathrm{upper}=30$;
$$\Pi(A)\propto\frac{1}{\log\left(A_\mathrm{upper}/A_\mathrm{lower}\right)}\begin{cases}1/A & \rm{if}\;A >A_\mathrm{lower}\;\rm{and}\;A <A_\mathrm{upper}\\0 &\rm{else}\end{cases}\,,$$
and for the duration the prior is uniform between lower/upper cutoffs $\tau_\mathrm{lower}=3$ and $\tau_\mathrm{upper}=30$;
$$\Pi(\tau)\propto \frac{1}{\tau_\mathrm{upper}-\tau_\mathrm{lower}}\begin{cases}1& \rm{if}\;\tau>1\;\rm{and}\;\tau<100\\0 &\rm{else}\end{cases}\,.$$

#### The Evidence: $P(\rm{data})\equiv Z$

This is the normalisation factor for the posterior,
$$ Z = \int\mathrm{d}\theta^{\mu} \; \mathcal{L}(\theta^{\mu})\Pi(\theta^{\mu}) \,. $$


---------

In the new notation, Bayes' theorem becomes
$$ P(\theta^{\mu} | \mathrm{data}) = \frac{\mathcal{L}(\theta^{\mu}) \Pi(\theta^{\mu})}{Z} \,. $$

## Section 2.2: CPNest Implementation

The **likelihood** and **prior** functions are the inputs to any Bayesian analysis. Here these functions are implemented inside the *model* class of the *CPNest* python package.

CPNest implements the nested sampling algorithm [J. Skilling (2006) doi:10.1214/06-BA127]. Nested sampling calculates $Z$ by Monte-Carlo integration of $\int\mathrm{d}\theta \; \mathcal{L}(\theta)\Pi(\theta)$. As by product, this algorithm produces a list parameter values $\{\theta_{1}, \theta_2, \ldots, \theta_{N_\rm{samples}}\}$ which are distributed according to the posterior.

In [ ]:
class CPNestModel(cpnest.model.Model):
    
    def __init__(self, catalog_data, event_num=0, model='A', sigma=0.1):
        """
        INPUTS
        ------
        catalog_data: numpy array
            the time series data for all light curves
        event_num: int
            which event to analyse? e.g. 1, 2,... [default to 1]
        model: string
            which model to use? e.g. 'A' or 'B' or 'Null' [defult to 'A']
        sigma: float
            the 1-sigma uncertainty on the measurements
        """
        
        self.catalog = catalog_data
        
        self.event = event_num
        
        self.model = model
        
        self.sigma = sigma
        
        if model is 'Null': # null hypothesis
            self.names   = []
            self.bounds  = []
            
        else: # signal hypotheses
            self.names   = ['amplitude', 'duration']
            self.bounds  = [[0.03, 30], [3, 30]]
    
    def log_prior(self, params):
        """
        The log-prior distribution

        INPUTS
        ------
        params: dict of model parameters
            keys 'amplitude' and 'duration'
    
        RETURNS
        -------
            LogPrior: float
        """
        if self.model is 'Null':
            return 0
        elif not self.in_bounds(params): 
            return -np.inf
        else:
            LogPrior = -np.log(np.log(self.bounds[0][1]/self.bounds[0][0]))
            LogPrior -= np.log(self.bounds[1][1]-self.bounds[1][0])
            return LogPrior 
        
    def log_likelihood(self, params):
        """
        The log-likelihood distribution
    
        LogLike = sum_i ( -0.5*(data_i-model_i)**2/sigma**2 ) - norm_const
    
        INPUTS
        ------
        params: dict of model parameters
            keys 'amplitude' and 'duration'
    
        RETURNS
        -------
            LogLike: float
        """
        times = self.catalog[:,0]
        data_light_curve = self.catalog[:,self.event+1]
    
        if self.model=='Null':
            model_lightcurve = np.zeros(len(times))
        elif self.model=='A':
            model_lightcurve = ModelA(times, params)
        elif self.model=='B':
            model_lightcurve = ModelB(times, params)

        LogLike = -0.5 * np.sum( (data_light_curve-model_lightcurve)**2 / self.sigma**2 )
        LogLike -= 0.5*len(times)*np.log(2*np.pi*self.sigma**2)
    
        return LogLike
    

## Section 2.2: Analysing First Event With Model A

Now we have set up the CPNest class it is a simple matter to create an instance of this class for a particular event (say, event number 0) and run the analysis.

In [ ]:
eventID = 0
modelA = CPNestModel(catalog_data, 
                     event_num=eventID, 
                     model='A',
                     sigma=sigma)

nest_modelA = cpnest.CPNest(modelA, 
                            nlive=1024, 
                            output='results/event'+str(eventID)+'_A', 
                            nthreads=4, 
                            verbose=1)

nest_modelA.run()
posterior_samplesA = nest_modelA.get_posterior_samples() 

### Section 2.2.1: Parameter Estimation

As mentioned above, the nested sampling algorithm produces (as a by product) as list of *posterior samples*. This is a list of parameter values, each of witch is an independant random draw from the posterior (AKA target distribution). We can visualise the posterior by simply plotting a histogram of the posterior samples.

In [ ]:
samples = np.array(list(zip(*[posterior_samplesA[name] for name in modelA.names])))
corner(samples, labels=modelA.names)
plt.show()

### Section 2.2.2: Detection

We can use the Bayesian evidence to decide whether or not there is a signal present.

First, compute the evidence for hypothesis A, defined as
$$Z_{A} = \int\mathrm{d}\theta^{\mu}\;\mathcal{L}_{A}(\theta^{\mu})\Pi(\theta^{\mu}) \,. $$ 
This has already been done above by CPNest.

Second, compute the evidence for the null hypothesis. The null hypothesis has no parameters (i.e. $\theta=\{\}$) so the evidence is simply given by the likelihood,

$$ Z_0 = \mathcal{L}_{0}() \,. $$

(This is an example of a Savage-Dickey density ratio.)

In [ ]:
model0 = CPNestModel(catalog_data, 
                     event_num=eventID, 
                     model='Null')

folder = 'results/event'+str(eventID)+'_0/'
Z0 = np.array([ model0.log_likelihood({}), 0])

print('Computed log_evidences: {}'.format(Z0[0]))
os.makedirs(folder, exist_ok=True); np.savetxt(folder+'evidence.txt', Z0)

Finally, the *odds ratio* between the two hypothesis is given by the ratio

$$ \mathcal{O}_{A0} = \frac{Z_A}{Z_0} \,. $$

If this ratio is larger than unity then there is evidence in favour of a signal.

In [ ]:
from Utils import GetEvidence

LogOddsRatio = GetEvidence(eventID, model='A') - GetEvidence(eventID, model='0')

print("log(O_A0) = ", LogOddsRatio)

This is a **huge** number; there is definitely a signal present. The null hypothesis is ruled out at extremely high significance.

### Section 2.2.3: Model Selection

We can also use the Bayesian evidence to decide between two competing signal models.

We compute the evidences for each model, $Z_A$ and $Z_B$, and form the *odds ratio*

$$ \mathcal{O}_{AB} = \frac{Z_A}{Z_B} \,. $$

In [ ]:
# Define the CPNest model
modelB = CPNestModel(catalog_data, 
                     event_num=eventID, 
                     model='B')

# Set up the nested sampling algorithm
nest_modelB = cpnest.CPNest(modelB, 
                            nlive=1024, 
                            output='results/event'+str(eventID)+'_B', 
                            nthreads=4, 
                            verbose=0)

# Run the algorithm
nest_modelB.run()
posterior_samplesB = nest_modelB.get_posterior_samples() 

In [ ]:
LogOddsRatio = GetEvidence(eventID, model='A') - GetEvidence(eventID, model='B')
print("log(O_AB) = ", LogOddsRatio)

There is very strong evidence in favour of model B.

Now that we have run CPNest for model B, we can look at the posterior distribution for the parameters $\theta^\mu$ under this model. The posterior on the parameters for model B is not the same as that for model A.

In [ ]:
corner(np.array(list(zip(*[posterior_samplesB[name] for name in modelB.names]))), 
       labels=modelB.names)
plt.show()

## Section 2.3: Repeat Analysis for all Events in Catalog 

I ran this earlier for all events. The results are in the repository.

In [ ]:
"""
eventID = 1

# Define the CPNest model A
modelA = CPNestModel(catalog_data,
                     event_num=eventID,
                     model='A')

nest_modelA = cpnest.CPNest(modelA,
                            nlive=1024,
                            output='results/event'+str(eventID)+'_A',
                            nthreads=4,
                            verbose=0)

nest_modelA.run()
post = nest_modelA.get_posterior_samples()

# Define the CPNest model B
modelB = CPNestModel(catalog_data,
                     event_num=eventID,
                     model='B')

nest_modelB = cpnest.CPNest(modelB,
                            nlive=1024,
                            output='results/event'+str(eventID)+'_B',
                            nthreads=4,
                            verbose=0)

nest_modelB.run()
post = nest_modelB.get_posterior_samples()

# The null hypothesis
model0 = CPNestModel(catalog_data,
                     event_num=eventID,
                     model='Null')

folder = 'results/event'+str(eventID)+'_0/'
Z0 = np.array([ model0.log_likelihood({}), 0])

print('Computed log_evidences: ({},)'.format(Z0))
os.makedirs(folder, exist_ok=True); np.savetxt(folder+'evidence.txt', Z0)
"""

print("Uncomment and run repeatedly with eventID 1,2,...,9")

## Section 2.4: Summary Plots for our mock catalog

You can load the posterior samples from any of the events in the catalog and plot a corner plot if you wish to see the posterior distributions.

Here I will load the evidences for each hypothesis and for each event ($3\, \mathrm{hypotheses }\times 10\,\mathrm{events}=30$ evidences in total) and plot these to see if we can reach any general conclusions.

In [ ]:
Zs = np.array([ [GetEvidence(eventID, model='0'), 
                 GetEvidence(eventID, model='A'), 
                 GetEvidence(eventID, model='B')] for eventID in range(10)])

logO_A0 = Zs[:,1] - Zs[:,0]
logO_BA = Zs[:,2] - Zs[:,1]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axes[0].errorbar(np.arange(Nevents), np.log(np.maximum(1,logO_A0)), 
                 xerr=0., yerr=0.0, uplims=logO_A0<1, marker='+', linestyle='',
                 color='red', label='log(O_A0)')
axes[0].errorbar(np.arange(Nevents), np.log(np.maximum(1,logO_BA)), 
                 xerr=0., yerr=0.0, uplims=logO_BA<1, marker='+', linestyle='',
                 color='blue', label='log(O_BA)')
axes[0].set_xlabel("Event ID", fontsize=BIGGER_SIZE)
axes[0].set_ylabel("Log ( Log Odds Ratio )", fontsize=BIGGER_SIZE)
axes[0].legend(loc='upper right', fontsize=MEDIUM_SIZE)

for N in range(Nevents):
    axes[1].plot(times, catalog_data[:,N+1], label='event '+str(N))  
axes[1].set_xlabel("Time [s]", fontsize=BIGGER_SIZE)
axes[1].set_ylabel("Flux", fontsize=BIGGER_SIZE)
axes[1].legend(loc='upper right', fontsize=MEDIUM_SIZE)

plt.tight_layout()
plt.show()

It seems that hypothesis B is generally prefered. These light curves are not Gaussian (with the possible exception of event 8 which is extremely faint in any case).

# Section 3: Hierarchical Bayesian Models

Inevitably, as we observe more events and the size of our catalog grows, we will stop just asking questions about individual events and start to ask questions about the population as a whole. The formalism of Bayesian analysis can be repurposed to address these questions as well.

In the above analysis a flat prior on the duration was used. But we might wonder whether this really reflects the population in our catalog. Certainly, looking at the light curves it seems that the durations are all clustered around a similar length. What is this average duration and how tight is the clustering?

## Section 3.1: General Approach

We could rerun the above analyses with a new prior on the parameters
$$ \Pi(\theta^\mu) \rightarrow \Pi(\theta^\mu|\lambda^{\alpha}) \,, $$
where the new *hyperparameters* $\lambda^{\alpha}$ describe the population.

Bayes' theorem for an individual event now reads
$$ P\big(\theta^\mu_i|\mathrm{data}_i, \lambda^\alpha \big) = \frac{P\big(\mathrm{data}_i|\theta^\mu\big)\Pi\big(\theta^\mu | \lambda^\alpha\big)}{Z_i(\lambda^\alpha)} \,, $$
for a fixed $i$ and where the $\lambda^\alpha$ are viewed as being fixed (constant). The normalising evidence now depends on the hyperparameters $\lambda^\alpha$.

Bayes' theorem one level up! Bayes' theorem applied to the population reads
$$ P\big(\lambda^\alpha|\{\mathrm{data}\} \big) = \frac{P\big(\{\mathrm{data}\}|\lambda^\alpha\big)\Pi\big(\lambda^\alpha\big)}{Z} \,, $$
where the $\lambda^\alpha$ are now our new *hyperparameters* and $\Pi(\lambda^{\alpha})$ are the *hyperpriors* on these parameters.
At this new, higher level the *hyperlikehood* is given by

$$ P(\lambda^{\alpha} | \{\mathrm{data}\}) = \prod_{i=1}^{N_\mathrm{events}}Z_i(\lambda^\alpha) \,. $$
which, by the definition of $Z_{i}(\lambda^\alpha)$ becomes
$$ P(\lambda^{\alpha} | \{\mathrm{data}\}) = \prod_{i=1}^{N_\mathrm{events}}\int\mathrm{d}\theta^\mu\; P(\mathrm{data}_i|\theta^{\mu})\Pi(\theta^\mu|\lambda^\alpha)\,. $$
The posterior samples which we have already obtained provide a computationally efficient method for evaluating this *hyperlikelihood*;
$$ P(\lambda^{\alpha} | \{\mathrm{data}\}) = \sum_{i=1}^{N_\mathrm{events}}\log\left( \sum_{a=1}^{N_\mathrm{samples}} \frac{\Pi(\theta^{\mu}_{a}| \lambda^\alpha)}{\Pi(\theta_a)} \right) \,. $$


---------

For completeness, we also give here the expressions for the *hyperevidence*, although it will not be needed;

$$ Z = \int\mathrm{d}\lambda\;P(\lambda^{\alpha} | \{\mathrm{data}\})\Pi(\lambda^{\alpha}) \,.$$


## Section 3.1: Modelling the Population - Specific example

We will choose as our model for the population (AKA *hyperprior*) to be
$$ \Pi(\theta^{\mu}|\lambda^{\nu}) = \Pi(A)\frac{\exp\left(\frac{-(\tau-\tau_0)^2}{2\Delta^2}\right)}{\sqrt{2\pi\Delta^2}}$$
where $\Pi(A)$ is the Jeffreys prior defined above and $\lambda^{\nu}=\{\tau_0, \Delta\}$.

In other words, we are modelling the population of transients as having durations $\tau$ which are log-normally distributed and we are trying to use the catalog of the first few events to measure the mean, $\tau_0$, and spread, $\Delta$, of the durations.

In [ ]:
catalog_samples = [np.loadtxt('results/event'+str(eventID)+'_B/posterior.dat') 
                   for eventID in range(Nevents)]

In [ ]:
class CPNestHyperModel(cpnest.model.Model):
    
    def __init__(self, catalog_samples):
        """
        INPUTS
        ------
        catalog_sample: list
            list length Nevents
            each entry in list is a np.array of the posterior samples
        """
        self.catalog_samples = catalog_samples
        
        self.names   = ['Tau0', 'Delta']
        self.bounds  = [[5, 15], [0.5, 2]] # Here I choose flat hyperpriors on 
                                           # the hyperparameters Tau0 and Delta and
                                           # relatively narrow ranges. 
            
    def log_prior(self, params):
        """
        The log-hyperprior distribution

        INPUTS
        ------
        params: dict of model parameters
            keys 'Tau0' and 'Delta'
        
        RETURNS
        -------
            LogPrior: float
        """
        if not self.in_bounds(params): 
            return -np.inf
        else:
            LogPrior = -np.log(self.bounds[0][1]-self.bounds[0][0])
            LogPrior -= np.log(self.bounds[1][1]-self.bounds[1][0])
            return LogPrior 
        
    def log_likelihood(self, params):
        """
        The log-hyperlikelihood distribution
    
        INPUTS
        ------
        params: dict of model parameters
            keys 'Tau0' and 'Delta'
           
        RETURNS
        -------
            LogLike: float
        """
        LogLike = 0.
        for samples in self.catalog_samples:
            LogLike += np.log(np.sum(
                norm.pdf(samples[:,1], loc=params['Tau0'], scale=params['Delta'])
            ) / (1./99.))
        
        return LogLike

Load the posterior samples for all of the individual events in the catalog.

In [ ]:
catalog_samples = [np.loadtxt('results/event'+str(eventID)+'_B/posterior.dat') 
                   for eventID in range(Nevents)]

Now we run the population analysis. We use the machinery of CPNest exactly as before, it is only the interpretation of the various quantities that has changed.

In [ ]:
hypermodel = CPNestHyperModel(catalog_samples)

nest_hypermodel = cpnest.CPNest(hypermodel, 
                            nlive=1024, 
                            output='results/population_analysis', 
                            nthreads=4, 
                            verbose=1)

nest_hypermodel.run()
nest_hypermodel_samples = nest_hypermodel.get_posterior_samples() 

In [ ]:
corner(np.array(list(zip(*[nest_hypermodel_samples[name] for name in hypermodel.names]))), 
       labels=hypermodel.names
       #, truths=[10, 1]
      )
plt.show()

As can be seen from the above corner plot, the events have mean duration of around $(9.4\pm 0.4)\,$s with a spread of $(1.2\pm0.3)\,$s. 

Because these are fake events which I generated myself, we can check that these results are consistent with a population I simulated (uncomment the "truths" line in the above cell).

# Section 4.0 Conclusions

See the lecture slides